# Use MPI and Scalapy to distribute all of MICA workflow to work on multiple nodes

prep.py component \
Read input file and slice into manageable sizes

In [1]:
from MICA.lib import utils

In [2]:
#from IPython import parallel
import ipyparallel as ipp

# Launch an ipython parallel cluster
Run this on hpc node to launch a cluster with mpi engines

In [3]:
#this currently needs to be launched from terminal
#import subprocess 
#We need to launch an ipython parallel cluster
#!ipcluster start --n=4
#subprocess.Popen(['ipcluster','start','--n=4'])
#subprocess.Popen(['ipcluster', 'start', '--engines=MPIEngineSetLauncher', '--log-level', 'DEBUG', '--n=4'])
#!ipcluster start --engines=MPIEngineSetLauncher --log-level DEBUG --n=4 &

In [4]:
# Create a parallel client so that we can use %%px cell magic
# With rc and dview, we can interact between mpi ranks and the thread running this notebook
rc = ipp.Client()
dview = rc[:]
rc.ids

[0, 1, 2, 3]

In [5]:
%%px
#load all necessary libraries onto each rank

from scipy.sparse import csr_matrix
from mpi4py import MPI
import sys
import numba
import pandas as pd
import scanpy as sc
import scipy as sci
import numpy as np
import anndata
import time
from sklearn.decomposition import PCA
import fast_histogram
import logging
logging.basicConfig(level=logging.INFO)
from MICA.lib import utils
from scalapy import *

In [6]:
%%px
import os
import socket
#from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()
#sys.stdout.write(
#    "Hello, World! I am process %d of %d on %s.\n" 
#    % (rank, size, name))

print("{host}[{pid}]: {rank}/{size}".format(
    host=socket.gethostname(),
    pid=os.getpid(),
    rank=comm.rank,
    size=comm.size,
))

[stdout:0] noderome178[9284]: 0/4
[stdout:1] noderome178[9285]: 1/4
[stdout:2] noderome178[9286]: 2/4
[stdout:3] noderome178[9287]: 3/4


In [7]:
%%px
#this is a defined elsewhere as a standalone python executable
#we are duplicating here to experiment with format for distrubuted computing
def prep_dist(input_file, out_name, slice_unit):
    """ Preprocess input file to create sliced matrices.

    Reads file into HDF5-format, adds parameters, slices data in file, generates several files with
    different combinations of slices.

    Args:
        input_file (str): path to input text-file
        out_name   (str): common rootname of generated output files
        slice_unit (int): size of each slice of cell data in input text-file
    """
    logging.basicConfig(level=logging.INFO)
    
    #Read in whole file stored in anndata csr format
    adf=utils.read_anndata_file(input_file)
    #if adf==None :
    #    raise Exception("Input file ",input_file," not found.")
        
    print("initial data size=",adf.shape)

    slice_size = int(slice_unit)
    
    #df = pd.HDFStore(df_file)["slice_0"]
    
    #compute number of slices needed to break dataset in to slice_size row blocks
    b = int(np.ceil(float(adf.shape[0]) / float(slice_size)))
    #determine how many digits are in b so we can pad spaces for the string output
    digit = int(np.floor(np.log10(b)) + 1)
    #loop over slice numbers
    for i in range(b):
        #slice name is equal to batch index
        slice_name = str(i).zfill(digit)
        #compute batch row indices
        start = i * slice_size
        end = np.min([(i + 1) * slice_size, adf.shape[0]])
        #copy slice to array of slices
        adf_sub=adf[start:end,:]
        #write to file so we don't have to keep each slice in memory
        output_file_name = out_name + ".slice_" + slice_name +".h5ad"
        print("output_file_name: ",output_file_name)
        adf_sub.write(output_file_name)
        
    #return nrows and nslices
    return adf.shape[0], adf.shape[1], b
    


In [8]:
%%px
#ceb create csr version of numba_histogram2d, also compute_bin with knowledge that minx will always be zero

@numba.jit(nopython=True)
def numba_nan_fill(x):
    shape = x.shape
    x = x.ravel()
    x[np.isnan(x)] = 0.0
    x = x.reshape(shape)
    return x

@numba.jit(nopython=True)
def numba_inf_fill(x):
    shape = x.shape
    x = x.ravel()
    x[np.isinf(x)] = 0.0
    x = x.reshape(shape)
    return x

@numba.jit(nopython=True, fastmath=True)
def compute_bin_upperbound(x, max, num_bins):
    """ Compute bin index for a give number.
        Assume that min is always zero
    """
    # special case to mirror NumPy behavior for last bin
    if x == max:
        return num_bins - 1 # a_max always in last bin

    bin = int(num_bins * x / max)

    if bin >= num_bins:
        return None
    else:
        return bin

@numba.jit(nopython=True, fastmath=True)
def numba_histogram2d_csr(arr1, cols1, arr2, cols2, ncols, num_bins):
    """ Compute the bi-dimensional histogram of two data samples.
    Args:
        arr1 (array_like, shape (N,)): An array containing the x coordinates of the points to be histogrammed.
        arr2 (array_like, shape (N,)): An array containing the y coordinates of the points to be histogrammed.
        num_bins (int): int
    Return:
        hist (2D ndarray)
    """
    #for csr arrays we have to compute zero bins ahead of time 
        
    bin_indices1 = np.zeros((ncols,), dtype=np.int16)
    max1 = arr1.max()
    #note that bin_indices has same size/indices as full array x and y
    for i, x in enumerate(arr1.flat):
        #assume zero min
        bin_indices1[cols1[i]] = compute_bin_upperbound(x, max1, num_bins)

    bin_indices2 = np.zeros((ncols,), dtype=np.int16)
    max2 = arr2.max()
    for i, y in enumerate(arr2.flat):
        #assume zero min
        bin_indices2[cols2[i]] = compute_bin_upperbound(y, max2, num_bins)

    hist = np.zeros((num_bins, num_bins), dtype=np.int16)
    for i, b in enumerate(bin_indices1):
        hist[b, bin_indices2[i]] += 1
        
    return hist

In [9]:
%%px
#import numba
@numba.jit(nopython=True, fastmath=True)
def numba_calc_mi_dis_csr(arr1, cols1, arr2, cols2, bins, m):
    """ Calculates a mutual information distance D(X, Y) = H(X, Y) - I(X, Y) using bin-based method

    It takes gene expression data from single cells, and compares them using standard calculation for
    mutual information and joint entropy. It builds a 2d histogram, which is used to calculate P(arr1, arr2).

    Args:
        arr1 (float) nparray of csr: gene expression data for cell 1
        cols1 (int): column indices for csr arr1
        arr2 (float) nparray of csr: gene expression data for cell 2
        cols2 (int): column indices for csr arr
        bins           (int): number of bins
        m              (int): number of genes
    Returns:
        a float between 0 and 1
    """
    hist = numba_histogram2d_csr(arr1, cols1, arr2, cols2, m, bins)
    
    sm = np.sum(hist, axis=1)
    tm = np.sum(hist, axis=0)
    sm = sm / float(sm.sum())
    tm = tm / float(tm.sum())

    sm_tm = np.zeros((bins, bins), dtype=np.float32)
    for i, s in enumerate(sm):
        for j, t in enumerate(tm):
            sm_tm[i, j] = s * t

    fq = hist / float(m)
    div = np.true_divide(fq, sm_tm)
    numba_nan_fill(div)
    ent = np.log(div)
    numba_inf_fill(ent)
    agg = np.multiply(fq, ent)
    #joint_ent = -np.multiply(fq, numba_inf_fill(np.log(fq))).sum()
    #return joint_ent - agg.sum()
    return agg.sum()

In [10]:
%%px
#numba compilation cannot interpret the creation of a 2d array inside of this function so we pass in and out SM_block instead of returning it
#import numba
@numba.jit(nopython=True, fastmath=True)
def process_matrices(Arows,Amat_data,Amat_indptr,Amat_indices,
                     Brows,Bmat_data,Bmat_indptr,Bmat_indices,
                     num_bins,num_genes,
                     SM_block, symmetric=False):
    #(mat1.n_obs,mat1.X.data,mat1.X.indptr,mat1.X.indices, mat2.n_obs,mat2.X.data,mat2.X.indptr,mat2.X.indices,SM,num_bins,mat2.n_vars,symmetric)

    #Arows = mat1.n_obs
    #Brows = mat2.n_obs
    #num_genes = mat1.n_vars
    
    #SM_block = np.ndarray(shape=(Arows,Brows))#, dtype=float, order='F')
    #SM_block = SM_block.reshape(Arows,Brows)
    for i in range(Arows):
        Arowstart = Amat_indptr[i]
        Arowend   = Amat_indptr[i+1]
        Arow_cols = Amat_indices[Arowstart:Arowend]
        Arow_data = Amat_data[Arowstart:Arowend]

        Bstart=0
        Bend=Brows
        #if(symmetric):Bstart=i #upper triangluar
        if(symmetric):Bend=i+1 #lower triangular
        for j in range(Bstart,Bend): 
            Browstart = Bmat_indptr[j]
            Browend   = Bmat_indptr[j+1]
            Brow_cols = Bmat_indices[Browstart:Browend]
            Brow_data = Bmat_data[Browstart:Browend]               
            SM_block[i,j] = numba_calc_mi_dis_csr(Arow_data, Arow_cols, Brow_data, Brow_cols, num_bins, num_genes)
            #SM_block[i*Arows+j] = numba_calc_mi_dis_csr(Arow_data, Arow_cols, Brow_data, Brow_cols, num_bins, num_genes)

            
    return #SM_block


In [11]:
%%px
#from mpi4py import MPI
from scipy.sparse import csr_matrix
#import sys
import pandas as pd

#create a 2d list to hold blocks of similarity matrix
#this should be a global var
#SM = [[None for j in range(b)] for i in range(b)] 

def calc_distance_metric_distributed(in_file_path, project_name, nrows, ncols, nslices, SM):
    
    """ Prepares the already sliced input file for further calculation in MICA.
    Enters pairs of slices (matrices) into temporary HDF5-format files. It enters them
    individually, using their unique key. It also enters the parameter data for every single 
    pair into the key "params", which consists of: [key1, key2, num_bins, num_genes,
    pair_index, project_name, num_slices]
    Args:
        in_file      (str): path to sliced HDF5-format file
        project_name (str): project name used to generate path for final outputs
        nrows : number of rows in global matrix
        ncols : number of vars in global matrix
    """

    
    #create a 2d list to hold blocks of similarity matrix
    #this should be a global var
    #SM = [[None for j in range(b)] for i in range(b)] 
    
    
    #input file is full input that has been segmented into b blocks of rows
    
    #nranks would ideally be equal to  b(b+1)/2
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    myrank = comm.Get_rank()
    name = MPI.Get_processor_name()
    #sys.stdout.write("Hello, World! I am process %d of %d on %s.\n" % (myrank, size, name))
    
    #in_ = pd.HDFStore(in_file, "r")  # slice.h5
    #b = int(np.ceil(float(nrows) / float(slice_size)))
    digit = int(np.floor(np.log10(nslices)) + 1)

    num_bins = int(np.floor(nrows ** (1 / 3.0)))  # number of bins
    #print("bins= ",bins)
    #b = in_["attr"].loc["slice", 0]  # number of sliced matrix
    b = nslices #number of row blocks (cells)
    m = ncols  # number of cols per row (genes)

    n_block_comparisons = int((b * (b + 1)) / 2)  # total number of row block comparisons needed to compute entire global similarity matrix
    n_jobs_per_rank = n_block_comparisons/size
    if (myrank == 0): print("block comparsons = %d. jobs per rank = %d\n" % (n_block_comparisons, n_jobs_per_rank))

    #build list of row block comparisons that current mpi rank will process
    myslices=[]
    for i in range(b):
        for j in range(i,b): # j in range [i,b]
            idx = int(i * b + j - (i * (i + 1)) / 2)
            targetrank = idx//n_jobs_per_rank
            if (targetrank == myrank): myslices.append((i,j))            

    #from list just generated, only do work assigned to your rank
    for index, tuple in enumerate(myslices):
        #print("tuple=",tuple)
        i=tuple[0] #block row
        j=tuple[1] #block col      

        #get 1st slice (row block) file
        slice_name = str(i).zfill(digit)
        ##ceb we only want to read this once per i,j combination
        input_file = in_file_path + project_name + ".slice_" + slice_name +".h5ad"
        #print("infile seg1: ",input_file)
        mat1 = utils.read_anndata_file(input_file)
    

        #get 2nd slice (row block) file
        slice_name = str(j).zfill(digit)
        input_file = in_file_path + project_name + ".slice_" + slice_name +".h5ad"
        #print("infile seg2: ",input_file)
        mat2 = utils.read_anndata_file(input_file)    

        #check to see if block comparison will result in a symmetric SM matrix
        # so that we can reduce the number of computations in half
        symmetric=False
        if i==j: symmetric=True
        
        print("rank: ",myrank," comparison between segs:",i," x ",j," symmetric=",symmetric)
            
        #compute distance metrics between row blocks
        
        Arows = mat1.n_obs
        Brows = mat2.n_obs
        num_genes = mat1.n_vars #we will assume Acols==Bcols==num_genes
        
        #need SM for each block pair    
        #creates local array of zeros and assigns to global 2d list
        #create matrix of zeros with row order indexing
        SM[i][j] = np.zeros(shape=(Arows,Brows), dtype = float, order = 'C')
 
        #This numba function cannot create a numpy array internally so we return SM[i,j] as a variable
        process_matrices(mat1.n_obs,mat1.X.data,mat1.X.indptr,mat1.X.indices, 
                         mat2.n_obs,mat2.X.data,mat2.X.indptr,mat2.X.indices,
                         num_bins, num_genes, SM[i][j],
                         symmetric #if i==j we can eliminate half of computations
                        )

        #convert to csr
        #we may want to assign this to scalapack distributed matrix here
        #SM[i][j]=csr_matrix(SM[i][j])

    return

In [12]:
%%px
import os
cwd=os.getcwd()
if rank==0:
    print(cwd)
    
data_file_path = cwd+'/test_data/inputs/10x/PBMC/3k/pre-processed/'
input_file_name = data_file_path + 'pbmc3k_preprocessed.h5ad'
project_name = 'pbmc3k'
output_file_name = data_file_path+project_name



[stdout:0] /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA


In [13]:
%%px
#set slice size (max size of row blocks)
slice_size = 500

In [14]:
%%px
if rank==0:
    print (input_file_name)

[stdout:0] /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k_preprocessed.h5ad


## Run Prep_dist() to split file into slices

In [15]:
%%px
#Run prep.py only on one processor to create the slice files

g_nrows=0 #global number of rows (cells)
ncols=0
nslices=0
if rank==0: 
    #for some unknown reason, running the prep_dist defined in MICA.utils stalls here, although they are identical
    g_nrows, ncols, nslices = prep_dist(input_file_name, output_file_name, slice_size)
    
# this uses the iparallel dview object to distribute these variables from the notebook thread to all mpi ranks
# but the ranks can't communicate back to the notebook thread
#dview.push(dict(nrows=nrows, ncols=ncols, nslices=nslices))

#broadcast resultant variables from root to the other ranks
g_nrows = comm.bcast(g_nrows, root=0)
ncols = comm.bcast(ncols, root=0)
nslices = comm.bcast(nslices, root=0)

[stdout:0] 
initial data size= (2496, 10499)
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_0.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_1.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_2.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_3.h5ad
output_file_name:  /research/rgs01/home/clusterHome/cburdysh/MICA_Project/MICA_distributed/MICA/test_data/inputs/10x/PBMC/3k/pre-processed/pbmc3k.slice_4.h5ad


[stderr:0] 
/home/cburdysh/.conda/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/cburdysh/.conda/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


In [16]:
%%px
if rank==0:
    print("global nrows, ncols, slices: ",g_nrows, ncols, nslices)

[stdout:0] global nrows, ncols, slices:  2496 10499 5


## Read in anndata preprocessed files (in distributed mode, by node number) and calculate distance metrics between all row pairs


In [46]:
%%px
#create a 2d list to hold blocks of similarity matrix
#this should be stored in a distributed scalapack matrix
b=nslices #row blocks
SM = [[None for j in range(b)] for i in range(b)] 

start = time.time()
calc_distance_metric_distributed(data_file_path, project_name, g_nrows, ncols, nslices, SM)
end = time.time()
print("Elapsed = %s" % (end - start))


[stdout:0] 
block comparsons = 15. jobs per rank = 3

rank:  0  comparison between segs: 0  x  0  symmetric= True
rank:  0  comparison between segs: 0  x  1  symmetric= False
rank:  0  comparison between segs: 0  x  2  symmetric= False
rank:  0  comparison between segs: 0  x  3  symmetric= False
Elapsed = 26.6834774017334
[stdout:1] 
rank:  1  comparison between segs: 0  x  4  symmetric= False
rank:  1  comparison between segs: 1  x  1  symmetric= True
rank:  1  comparison between segs: 1  x  2  symmetric= False
rank:  1  comparison between segs: 1  x  3  symmetric= False
Elapsed = 26.795007944107056
[stdout:2] 
rank:  2  comparison between segs: 1  x  4  symmetric= False
rank:  2  comparison between segs: 2  x  2  symmetric= True
rank:  2  comparison between segs: 2  x  3  symmetric= False
rank:  2  comparison between segs: 2  x  4  symmetric= False
Elapsed = 26.59738779067993
[stdout:3] 
rank:  3  comparison between segs: 3  x  3  symmetric= True
rank:  3  comparison between segs: 3 

In [47]:
#%%px 
###from scipy.sparse import csr_matrix #may not use csr as it complicates copy to distributed scalapack and is not used in scalapack apparently
#import collections
#for i in range(b):
#    for j in range(i,b):
#        if isinstance(SM[i][j], collections.Iterable):
#            #print("Rank:",rank, " SM[",i,"][",j,"]=",SM[i][j])
#            print("SM[",i,"][",j,"]=",SM[i][j],"\n")

## Create distributed matrix for scalapack and copy distributed blocks into object

## copy lower triangular transpose to upper triangular for diagonal blocks

In [48]:
%%px 
##from scipy.sparse import csr_matrix #may not use csr as it complicates copy to distributed scalapack and is not used in scalapack apparently
import collections
for i in range(b):
    for j in range(i,b):
        if isinstance(SM[i][j], collections.Iterable):
            if i==j: #copy lower triangular transpose to upper triangular 
                for ii in range(SM[i][j].shape[0]):
                    for jj in range(ii+1,SM[i][j].shape[1]):
                        (SM[i][j])[ii,jj]=(SM[i][j])[jj,ii]
                #print("Rank:",rank, " SM[",i,"][",j,"]=",SM[i][j])

## Populate a global array with all of the MI data from each rank

Preferably, we would like each rank to contribute of their block MI matrices to the global matrix,
but currently the distributed global matrix has to be constructed from a global (not distributed) array

First, each rank loads it's matrix contribution into a global block matrix, \
And then we use MPI to transfer these contributions to rank 0 from all other ranks.

In [49]:
#copy SM data into global distributed matrix and then write to file?

#then we can read that file into the Scalapack block cyclic matrix form

In [50]:
%%px
#test to distribute matrix from local blocks rather than global array
from scalapy import blacs
import os
import numpy as np
import scipy.linalg as la
from mpi4py import MPI
from scalapy import core
import scalapy.routines as rt

#distribute MI components to ranks as scalapack distributed matrix
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size #total number of ranks

global_num_rows =g_nrows
global_num_cols =g_nrows
local_num_rows =g_nrows/b

block_size=64 #default is 32

#Define process grid with process rows and process cols
#We'll use a 2d process grid to distribute blocks so we want to have num_ranks divisible by 2
assert((size % 2)==0)
#ideally we would like BR and BC to the square root of the num_ranks to get a square process matrix
PR=int(np.sqrt(size))
PC=PR

#if we can't create a square matrix, get next best dimensions
if PR*PR!=size:
    PC=size//PR
if rank==0:
    print("PR=",PR, "PC=",PC)

#sets default context and block_shape
core.initmpi([PR, PC],block_shape=[block_size,block_size])
#convert to fortran array indexing to match scalapack functions
#create global matrix from array on rank0
dMI2=core.DistributedMatrix(global_shape=[g_nrows,g_nrows],dtype=np.float64)


[stdout:0] PR= 2 PC= 2


In [51]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI2 = %s' % (rank, dMI2.global_shape))
#print ('rank %d has local_shape of dMI2 = %s' % (rank, dMI2.local_shape))
#print ('rank %d has block_shape of dMI2 = %s' % (rank, dMI2.block_shape))
##print(dMI2.local_array)

In [52]:
#%%px
#blocksize=slice_size
#testrank=3
#testmat=np.zeros(shape=(4,4))
#if comm.rank==testrank:
#    testmat=SM[3][3]
#    #testmat=np.zeros(shape=(500,500))
#    s_block_shape=np.shape(testmat)
#else:
#    testmat=np.zeros(shape=(4,4))
#    s_block_shape=np.shape(testmat)

#s_block_shape = comm.bcast(s_block_shape, root=testrank)   
#copy_from_np(dMI2, testmat, asrow=0, anrow=None, ascol=0, ancol=None, srow=0, scol=0, block_shape=s_block_shape, rank=testrank) #all ranks works


## Copy each SM block submatrix to distributed block cyclic matrix

In [53]:
%%px
blocksize=slice_size
n_jobs_per_rank= (int((b * (b + 1)) / 2))/comm.Get_size()
import collections

for i in range(b):
    for j in range(i,b): # j in range [i,b]
        idx = int(i * b + j - (i * (i + 1)) / 2)
        srank = idx//n_jobs_per_rank
        lA=np.zeros(shape=(2,2))
        s_block_shape=np.shape(lA)
        if isinstance(SM[i][j], collections.Iterable):
            lA=SM[i][j]
            s_block_shape=np.shape(lA)
            #print("copy SM[",i,j,"] shape: ",s_block_shape)
        #broadcast sending ranks block shape to all
        s_block_shape = comm.bcast(s_block_shape, root=srank)   
        dMI2.np2self(lA, srow=i*blocksize, scol=j*blocksize, block_shape=s_block_shape, rank=srank )      

In [54]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI2 = %s' % (rank, dMI2.global_shape))
#print ('rank %d has local_shape of dMI2 = %s' % (rank, dMI2.local_shape))
#print ('rank %d has block_shape of dMI2 = %s' % (rank, dMI2.block_shape))
#print(dMI2.local_array)

## copy transpose of blocks to fill upper triangular distributed matrix

In [55]:
%%px
blocksize=slice_size
n_jobs_per_rank= (int((b * (b + 1)) / 2))/comm.Get_size()
import collections

for i in range(b):
    for j in range(i+1,b): # j in range [i,b]
        idx = int(i * b + j - (i * (i + 1)) / 2)
        srank = idx//n_jobs_per_rank
        lA=np.zeros(shape=(2,2))
        s_block_shape=np.shape(lA)
        if isinstance(SM[i][j], collections.Iterable):
            lA=np.transpose(SM[i][j])
            s_block_shape=np.shape(lA)
            #print("copy SM[",j,i,"] shape: ",s_block_shape)
        #broadcast sending ranks block shape to all
        s_block_shape = comm.bcast(s_block_shape, root=srank)   
        dMI2.np2self(lA, srow=j*blocksize, scol=i*blocksize, block_shape=s_block_shape, rank=srank )      

In [24]:
## need to also fill in empty symmetric upper triangular portion

In [ ]:
# Even though this is a symmetric matrix, for further processing, we need to copy block data to rest of matrix

In [56]:
%%px
if rank==0: 
    print ('rank %d has global_shape of dMI2 = %s' % (rank, dMI2.global_shape))
print ('rank %d has local_shape of dMI2 = %s' % (rank, dMI2.local_shape))
print ('rank %d has block_shape of dMI2 = %s' % (rank, dMI2.block_shape))
#print(dMI2.local_array[0:20,0:20])
print(dMI2.local_array)

[stdout:0] 
rank 0 has global_shape of dMI2 = (2496, 2496)
rank 0 has local_shape of dMI2 = (1280, 1280)
rank 0 has block_shape of dMI2 = (64, 64)
[[0.33805914 0.066332   0.05676848 ... 0.05752099 0.04267657 0.0668972 ]
 [0.066332   0.45212405 0.06047588 ... 0.06336014 0.04781611 0.06866279]
 [0.05676848 0.06047588 0.39173321 ... 0.05280795 0.04746441 0.05657429]
 ...
 [0.05752099 0.06336014 0.05280795 ... 0.28514319 0.05104461 0.05835302]
 [0.04267657 0.04781611 0.04746441 ... 0.05104461 0.21451758 0.04515555]
 [0.0668972  0.06866279 0.05657429 ... 0.05835302 0.04515555 0.31753523]]
[stdout:1] 
rank 1 has local_shape of dMI2 = (1280, 1216)
rank 1 has block_shape of dMI2 = (64, 64)
[[0.04636978 0.05072926 0.05500222 ... 0.05035315 0.06835701 0.06148071]
 [0.05683409 0.05903542 0.05992358 ... 0.06264001 0.07618326 0.06856185]
 [0.04752714 0.04735661 0.05504157 ... 0.07159437 0.06133829 0.05748736]
 ...
 [0.04205094 0.0458999  0.04639795 ... 0.0493155  0.05951592 0.05640946]
 [0.0368094 

In [57]:
#redistribute for 

In [ ]:
#%%px
###lr=[range(4)]
###lc=[range(4)]
####Check to see of data was transferred to rank 0
#if rank==0:
#    for i in range(b):
#            for j in range(i,b): # j in range [i,b]
#                #print("SM[",i,j,"] ",np.shape(SM[i][i]))
#                print("SM[",i,j,"] ",SM[i][j])
##                print( (SM[i][j])[lr,lc] )   

In [ ]:
#%%px
#from inspect import getmembers, isfunction, ismodule
#if rank == 0:
#    print([o[0] for o in getmembers(scalapy) if ismodule(o[1])])

In [ ]:
#%%px
#from inspect import getmembers, isfunction, ismodule
#import scalapy
#if rank==0:
#    print(getmembers(scalapy.blacs, isfunction))
#    #print(getmembers(scalapy.routines, isfunction))

In [ ]:
#%%px
#total number of global blocks = total number of block comparisons
#Should be greater than number of ranks to improve load balancing
# b is number of slices (blocks of rows) original data has been discretized into.
#The size of these blocks can be variable
#global_number_of_matrix_blocks= int((b * (b + 1)) / 2) 

#We'll use a 2d process grid to distribute blocks so we want to have num_ranks divisivle by 2

In [ ]:
#bcast SM[i][j] from each rank to root rank 0 so that we can load global matrix array


## Convert array into distributed block cyclic global matrix
Simply load a numpy vector with data from each block matrix to use in scalapak distributed matrix initialization

In [59]:
%%px
#Write MI matrix to file
mi_filename = data_file_path+project_name+'_mi_distributed.scalapack'
dMI2.to_file(mi_filename)

In [ ]:
#%%px
#Read MI matrix from file
#mi_filename = data_file_path+project_name+'_mi_distributed.scalapack'
#dMI.from_file(mi_filename, global_shape=[g_nrows,g_nrows], dtype=np.float64, block_shape=[block_size,block_size])

In [61]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI = %s' % (rank, dMI2.global_shape))
#print ('rank %d has local_shape of dMI = %s' % (rank, dMI2.local_shape))
#print ('rank %d has block_shape of dMI = %s' % (rank, dMI2.block_shape))
#print(dMI2.local_array)

## Now we need to create a matrix with the diagonal as the first column

In [ ]:
%%px
#Array must be 2D for this to work
#convert to fortran array indexing to match scalapack functions
global_diag=np.asfortranarray(global_diag)
#create global matrix from array on rank0
dMI_diag=core.DistributedMatrix.from_global_array(global_diag,rank=0)

In [ ]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI_diag = %s' % (rank, dMI_diag.global_shape))
#print ('rank %d has local_shape of dMI_diag = %s' % (rank, dMI_diag.local_shape))
#print ('rank %d has block_shape of dMI_diag = %s' % (rank, dMI_diag.block_shape))
##print(dMI_diag.local_array)

In [ ]:
#%%px
#dMI_diag_T=dMI_diag.T
#if rank==0: 
#    print ('rank %d has global_shape of dMI_diag = %s' % (rank, dMI_diag_T.global_shape))
#print ('rank %d has local_shape of dMI_diag = %s' % (rank, dMI_diag_T.local_shape))
#print ('rank %d has block_shape of dMI_diag = %s' % (rank, dMI_diag_T.block_shape))
#
#print(dMI_diag_T.local_array)

## Use scalapack to compute distributed GEMM

In [ ]:
%%px
import scalapy.routines as rt
#dMI_norm=dMI_diag.T*dMI_diag
dMI_norm = rt.dot(dMI_diag,dMI_diag,transA='T')


In [ ]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI_diag = %s' % (rank, dMI_norm.global_shape))
#print ('rank %d has local_shape of dMI_diag = %s' % (rank, dMI_norm.local_shape))
#print ('rank %d has block_shape of dMI_diag = %s' % (rank, dMI_norm.block_shape))
#print(dMI_norm.local_array)

In [ ]:
%%px
#compute sqrt of each element
dMI_norm_square=core.DistributedMatrix.empty_like(dMI)
dMI_norm_square.local_array[:] = np.sqrt(dMI_norm.local_array[:])

In [ ]:
#%%px
#if rank==0: 
#    print ('rank %d has global_shape of dMI_diag = %s' % (rank, dMI_norm_square.global_shape))
#print ('rank %d has local_shape of dMI_diag = %s' % (rank, dMI_norm_square.local_shape))
#print ('rank %d has block_shape of dMI_diag = %s' % (rank, dMI_norm_square.block_shape))
#print(dMI_norm_square.local_array)

In [ ]:
%%px
dMI_normed=core.DistributedMatrix.empty_like(dMI)
dMI_normed.local_array[:] = dMI.local_array[:] / dMI_norm_square.local_array[:]

In [ ]:
%%px
if rank==0: 
    print ('rank %d has global_shape of dMI_diag = %s' % (rank, dMI_normed.global_shape))
print ('rank %d has local_shape of dMI_diag = %s' % (rank, dMI_normed.local_shape))
print ('rank %d has block_shape of dMI_diag = %s' % (rank, dMI_normed.block_shape))
print(dMI_normed.local_array)

In [ ]:
%%px
mi_normed_filename = data_file_path+project_name+'_mi_normed_distributed.scalapack'
dMI_normed.to_file(mi_normed_filename)

## Now compute eigenvalues and eigenvectors of dissimmilarity matrix

In [ ]:
#def mds(in_mat_file, max_dim, out_file_name, perplexity=30, print_plot="True", dist_method="mi",):
#    hdf = pd.HDFStore(in_mat_file)
#    if dist_method == "mi":
#        dlplf = 1 - hdf["norm_mi"]
#    elif dist_method == "euclidean":
#        df = hdf[dist_method]
#    else:
#        df = 1 - hdf[dist_method]
#
#    hdf.close()
#    n = df.shape[0]
#    H = np.eye(n) - np.ones((n, n)) / n
#    B = -H.dot(df ** 2).dot(H) / 2
#    evals, evecs = eigh(B, eigvals=(n - np.min([n, 200]), n - 1))
#    
#    idx = np.argsort(evals)[::-1]
#    evals = evals[idx]
#    evecs = evecs[:, idx]
#    evals_pos = evals > 0
#    L = np.diag(np.sqrt(evals[evals_pos]))
#    V = evecs[:, evals_pos]
#    Y = pd.DataFrame(
#        data=V.dot(L),
#        index=df.index,
#        columns=["mds_" + str(x) for x in np.arange(1, L.shape[0] + 1)],
#    )
#
#    Y.to_hdf(out_file_name + "_reduced.h5", "mds")  # save reduced mi in mds
#
#    if print_plot == "True":
#        vis = tsne(
#            Y,
#            max_dim,
#            out_file_name,
#            "mds",
#            perplexity,
#            print_plot,
#        )
#        vis.to_hdf(out_file_name + "_reduced", "mds_tsne")  # save preview in key "mds_tsne"

In [ ]:
%%px
import scalapy.routines as rt

n= g_nrows

#convert similarity matrix to dissimilarity matrix
#df= 1-df
MDS= core.DistributedMatrix.empty_like(dMI)
MDS.local_array[:]=1.0-dMI_normed.local_array[:]

# H = I-Ones/n
I= core.DistributedMatrix.identity(n=g_nrows)
Ones= core.DistributedMatrix.empty_like(dMI)
Ones.local_array[:]=1.0/n
H = core.DistributedMatrix.empty_like(dMI)
H.local_array[:] = I.local_array[:] - Ones.local_array[:]

# B = -H.dot(MDS**2).dot(H)/2
negH= core.DistributedMatrix.empty_like(dMI)
negH.local_array[:]= -H.local_array[:]
MDS2= core.DistributedMatrix.empty_like(dMI)
MDS2.local_array[:] = MDS.local_array[:]**2
C=rt.dot(negH,MDS2)
B = rt.dot(C,H)
B.local_array[:]=B.local_array[:]/2.0
#dMI_norm=dMI_diag.T*dMI_diag
#dMI_norm = rt.dot(dMI_diag,dMI_diag,transA='T')


In [ ]:
%%px
if rank==0: 
    print ('rank %d has global_shape of dMI_diag = %s' % (rank, MDS.global_shape))
print ('rank %d has local_shape of dMI_diag = %s' % (rank, MDS.local_shape))
print ('rank %d has block_shape of dMI_diag = %s' % (rank, MDS.block_shape))
print(MDS.local_array)

In [ ]:
%%px
if rank==0: 
    print ('rank %d has global_shape of dMI_diag = %s' % (rank, MDS2.global_shape))
print ('rank %d has local_shape of dMI_diag = %s' % (rank, MDS2.local_shape))
print ('rank %d has block_shape of dMI_diag = %s' % (rank, MDS2.block_shape))
print(MDS2.local_array)

In [ ]:
%%px
if rank==0: 
    print ('rank %d has global_shape of dMI_diag = %s' % (rank, B.global_shape))
print ('rank %d has local_shape of dMI_diag = %s' % (rank, B.local_shape))
print ('rank %d has block_shape of dMI_diag = %s' % (rank, B.block_shape))
print(B.local_array)

In [ ]:
%%px
import scalapy.routines as rt
#compute eigh(B,)
#we want to pick out the top 200 eigenvalues/vectors from the matrix
#evals, evecs = eigh(B, eigvals=(n - np.min([n, 200]), n - 1))
evals, dZd = rt.eigh(B,overwrite_a=False)
evecs = dZd.to_global_array(rank=0)
#gZd = dZd.to_global_array(rank=0)

In [ ]:
%%px
print(evals)

In [ ]:
#%%px
#import scalapy.routines as rt
##obtain dissimilarity matrix
#n= g_nrows
##MDS=1-dMI
#MDS= core.DistributedMatrix.empty_like(dMI)
##MDS.local_array[:]=1.0-dMI.local_Array[:]
#I= core.DistributedMatrix.identity(n=g_nrows)
#Ones= core.DistributedMatrix.empty_like(dMI)
#Ones.local_array[:]=1.0/n
##create arrays
##create identity matrix and then subtract 1/n from all entries
#H= np.eye(n) - np.ones((n, n)) / n
#H= I - Ones
#B= -H.dot(df ** 2).dot(H) / 2
#
##compute eigh(B,)
##we want to pick out the top 200 eigenvalues/vectors from the matrix
##evals, evecs = eigh(B, eigvals=(n - np.min([n, 200]), n - 1))
#evals, dZd = rt.eigh(B,overwrite_a=False)
#evecs = dZd.to_global_array(rank=0)

In [ ]:
#%%px
#dgi, dlri, dlci = diagMI.local_diagonal_indices()

In [ ]:
#%%px
#gi, lri, lci = dMI.local_diagonal_indices()

In [ ]:
#%%px
#diagMI.local_array[lri,0] = dMI.local_array[lri,lci]

In [ ]:
#%%px
#print ('rank %d has local_shape of dMI = %s' % (rank, diagMI.local_shape))
#print ('rank %d has block_shape of dMI = %s' % (rank, diagMI.block_shape))
#print(diagMI.local_array)

In [ ]:
#%%px
#diagMI_dot = dot(dMI_diag,dMI_diag,transB='T')
#diagMI_dot = dMI_diag.T * dMI_diag

#diagMI_dot.local_array[lri,lci] = 1.0/np.sqrt(diagMI_dot.local_array[lri,lci])

In [ ]:
#%%px
#print(diagMI_dot.local_array)

In [ ]:
#%%px
#print(diagMI_dot.local_array)

In [ ]:
#%%px
#diagMI_normed = dMI*diagMI_dot

In [ ]:
#%%px
#print(dMI_normed.local_array)

In [ ]:
#%%px
##lr=[range(2400,2495)]
##lc=[range(2400,2495)]
#print(len(dMI.local_array[lri,lci]))
#diag=dMI.local_array[lri,lci]
#print(diag)

In [ ]:
#%%px
#dMI_dot=dot(dMI,dMI,transB='T')

In [ ]:
#%%px
#dMI_dot.desc

In [ ]:
#%%px
#dgi, dlri, dlci = dMI_dot.local_diagonal_indices()
#print(dMI_dot.local_array[dlri,dlci])

In [ ]:
#%%px
#Note these values are in mixed order due to block cyclic partition. last rank last block does not correspond to last values in global diagonal
#print(diag)


In [ ]:
%%px
if rank==0:
    print(evals)

In [ ]:
%%px
if rank==0:
    nnzgZd=
    print(len(gZd))
    print(gZd)

In [ ]:
#compute eigenvalues,vectors
#evals1, evecs1 = scalapy.eigh(dMI, overwrite_a=False)

## Write distributed matrix to files

In [ ]:
## test function to sort out copy_from_np
%%px
from mpi4py import MPI
from scalapy import *

def _chk_2d_size(shape, positive=True):
    # Check that the shape describes a valid 2D grid. Zero shape not allowed when positive = True.
    if len(shape) != 2:
        return False
    if positive:
        if shape[0] <= 0 or shape[1] <= 0:
            return False
    else:
        if shape[0] < 0 or shape[1] < 0:
            return False
    return True


def copy_from_np(dmat, a, asrow=0, anrow=None, ascol=0, ancol=None, srow=0, scol=0, block_shape=None, rank=0):
        ## copy a section of a numpy array a[asrow:asrow+anrow, ascol:ascol+ancol] to self[srow:srow+anrow, scol:scol+ancol], once per block_shape

        Nrow, Ncol = dmat.global_shape
        srow = srow if srow >= 0 else srow + Nrow
        srow = max(0, srow)
        srow = min(srow, Nrow)
        scol = scol if scol >= 0 else scol + Ncol
        scol = max(0, scol)
        scol = min(scol, Ncol)
        if dmat.context.mpi_comm.rank == rank:
            if not (a.ndim == 1 or a.ndim == 2):
                raise ScalapyException('Unsupported high dimensional array.')

            a = np.asfortranarray(a.astype(dmat.dtype)) # type conversion
            a = a.reshape(-1, a.shape[-1]) # reshape to two dimensional
            am, an = a.shape
            asrow = asrow if asrow >= 0 else asrow + am
            asrow = max(0, asrow)
            asrow = min(asrow, am)
            ascol = ascol if ascol >= 0 else ascol + an
            ascol = max(0, ascol)
            ascol = min(ascol, an)
            m = am - asrow if anrow is None else anrow
            m = max(0, m)
            m = min(m, am - asrow, Nrow - srow)
            n = an - ascol if ancol is None else ancol
            n = max(0, n)
            n = min(n, an - ascol, Ncol - scol)
        else:
            m, n = 1, 1

        asrow = dmat.context.mpi_comm.bcast(asrow, root=rank)
        ascol = dmat.context.mpi_comm.bcast(ascol, root=rank)
        #print("asrow ascol=",asrow,ascol)
        #get sending ranks size
        m = dmat.context.mpi_comm.bcast(m, root=rank) # number of rows to copy
        n = dmat.context.mpi_comm.bcast(n, root=rank) # number of columes to copy
        #print("m,n=",m,n)
        if m == 0 or n == 0:
            return dmat

        block_shape = dmat.block_shape if block_shape is None else block_shape
        if not _chk_2d_size(block_shape):
            raise ScalapyException("Invalid block_shape")
        #print("block_shape=",block_shape)
        bm, bn = block_shape
        br = blockcyclic.num_blocks(m, bm) # number of blocks for row
        bc = blockcyclic.num_blocks(n, bn) # number of blocks for column
        rm = m - (br - 1) * bm # remained number of rows of the last block
        rn = n - (bc - 1) * bn # remained number of columes of the last block
        #print("bm,bn=",bm,bn)

        # due to bugs in scalapy, it is needed to first init an process context here
        ProcessContext([1, dmat.context.mpi_comm.size], comm=dmat.context.mpi_comm) # process context
        #print("br,bc=",br,bc)
        
        for bri in range(br):
            M = bm if bri != br - 1 else rm
            for bci in range(bc):
                N = bn if bci != bc - 1 else rn
                if dmat.context.mpi_comm.rank == rank:
                    #print("main rank=",dmat.context.mpi_comm.rank," bm bn: ",bm,bn," bri bci: ",bri,bci,"srow scol: ",srow,scol)
                    pc = ProcessContext([1, 1], comm=MPI.COMM_SELF) # process context
                    desc = dmat.desc
                    desc[1] = pc.blacs_context #new context containing only this rank
                    desc[2], desc[3] = a.shape
                    desc[4], desc[5] = a.shape
                    desc[8] = a.shape[0]
                    #print("descA= ",desc)
                    #print("descB= ",dmat.desc)

                    args = [M, N, a,                              asrow+1+bm*bri, ascol+1+bn*bci, desc,   
                            dmat._local_array, srow+1+bm*bri, scol+1+bn*bci, dmat.desc, dmat.context.blacs_context]
                else:#these processes do not own submatrix, but must still call function
                    #print("other rank=",dmat.context.mpi_comm.rank," bm bn: ",bm,bn," bri bci: ",bri,bci,"srow scol: ",srow,scol)
                    desc = np.zeros(9, dtype=np.int32)
                    desc[1] = -1           
                    #print("descA= ",desc)
                    #print("descB= ",dmat.desc)
                    args = [M, N, np.zeros(1, dtype=dmat.dtype) , asrow+1+bm*bri, ascol+1+bn*bci, desc,   
                            dmat._local_array, srow+1+bm*bri, scol+1+bn*bci, dmat.desc, dmat.context.blacs_context]
                from scalapy import lowlevel as ll
                call_table = {'S': (ll.psgemr2d, args),
                              'D': (ll.pdgemr2d, args),
                              'C': (ll.pcgemr2d, args),
                              'Z': (ll.pzgemr2d, args)}
                func, args = call_table[dmat.sc_dtype]
                #print (args)
                func(*args)
        return dmat

In [ ]:
##formerly used to copy to a global array
%%px
#copy SM[i][j] from each rank to root rank 0
n_jobs_per_rank= (int((b * (b + 1)) / 2))/comm.Get_size()
import collections
for i in range(b):
    for j in range(i,b): # j in range [i,b]
        idx = int(i * b + j - (i * (i + 1)) / 2)
        srank = idx//n_jobs_per_rank
        if rank!=0 and isinstance(SM[i][j], collections.Iterable):
            #print("send[",i,j,"] tag: ",idx)
            comm.send(SM[i][j],dest=0,tag=idx)
        elif rank==0 and not isinstance(SM[i][j], collections.Iterable):
            #MB=np.zeros(shape=(Arows,Brows), dtype = float, order = 'C')
            MB=comm.recv( source=srank, tag=idx)
            SM[i][j]=MB
            #print("recv[",i,j,"] tag: ",idx," shape: ",MB.shape, SM[i][j])
            

In [ ]:
#create and fill distributed matrix from global array
%%px
from scalapy import blacs
import os
import numpy as np
import scipy.linalg as la
from mpi4py import MPI
from scalapy import core
import scalapy.routines as rt

#distribute MI components to ranks as scalapack distributed matrix
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size #total number of ranks

global_num_rows =g_nrows
global_num_cols =g_nrows
local_num_rows =g_nrows/b

block_size=64 #default is 32

#Define process grid with process rows and process cols
#We'll use a 2d process grid to distribute blocks so we want to have num_ranks divisible by 2
assert((size % 2)==0)
#ideally we would like BR and BC to the square root of the num_ranks to get a square process matrix
PR=int(np.sqrt(size))
PC=PR

#if we can't create a square matrix, get next best dimensions
if PR*PR!=size:
    PC=size//PR
if rank==0:
    print("PR=",PR, "PC=",PC)

#create mpi comm context to send to blacs communicator    
#process context
#do I need this?
#cntxt = blacs.sys2blacs_handle(comm)
#initialize a process grid to distribute matrix blocks to.
#blacs.gridinit(ctxt,order='R',BR,BC)
#blacs.gridinit(pc,order='R',BR,BC)
#core.initmpi([PR, PC], block_shape=[local_num_rows,local_num_rows])
#instead of creating a global array on every processor, we want to load a 
#global array with local data
#MI=core.DistributedMatrix(global_shape=[global_num_rows,global_num_cols])


core.initmpi([PR, PC],block_shape=[block_size,block_size])

#convert to fortran array indexing to match scalapack functions
global_MI=np.asfortranarray(global_MI)
#create global matrix from array on rank0
dMI=core.DistributedMatrix.from_global_array(global_MI,rank=0)

#Array must be 2D, so this fails
#convert to fortran array indexing to match scalapack functions
#global_diag=np.asfortranarray(global_diag)
#create global matrix from array on rank0
#dMI_diag=core.DistributedMatrix.from_global_array(global_diag,rank=0)




#blacs.gridinit(ctxt, b, b)
#ranklist = [(0, 0), 
#            (0, 1), (1, 1),
#            (0, 2), (1, 2), (2, 2),
#            (0, 3), (1, 3), (2, 3), (3, 3),
#            (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)
#           ]